### Uncalibrated 3D Reconstruction and Evaluation

In [18]:
import numpy as np
import cv2 as cv
import open3d as o3d
from numba import njit, prange
from scipy.optimize import minimize_scalar
import os
import random
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors

# ----------------------
# Utility Functions
# ----------------------
@njit(parallel=True)
def fast_depth_filter(points):
    valid = np.empty(len(points), dtype=np.bool_)
    for i in prange(len(points)):
        z = points[i, 2]
        valid[i] = 0.2 < z < 50.0
    return valid

def get_colors(img, points_2d):
    h, w = img.shape[:2]
    x = np.clip(points_2d[:, 0].round(), 0, w - 1).astype(int)
    y = np.clip(points_2d[:, 1].round(), 0, h - 1).astype(int)

    if len(img.shape) == 2:  # Grayscale
        colors = img[y, x]
        return np.stack([colors, colors, colors], axis=1) / 255.0
    else:  # Color image
        colors = img[y, x]
        return colors[:, ::-1] / 255.0

# ----------------------
# Evaluation Metrics
# ----------------------
def compute_epipolar_error(ptsL, ptsR, F):
    ones = np.ones((len(ptsL), 1))
    ptsL_h = np.hstack((ptsL, ones))
    ptsR_h = np.hstack((ptsR, ones))
    errors = np.abs(np.diag(ptsR_h @ F @ ptsL_h.T))
    return np.mean(errors), errors

def compute_point_cloud_metrics(pcd):
    points = np.asarray(pcd.points)
    if len(points) < 2:
        return {
            'mean_neighbor_distance': 0,
            'depth_distribution': {'mean': 0, 'std': 0, 'min': 0, 'max': 0},
            'num_points': len(points)
        }

    nn = NearestNeighbors(n_neighbors=2).fit(points)
    distances, _ = nn.kneighbors(points)
    mean_nn_dist = np.mean(distances[:, 1])

    z_values = points[:, 2]
    depth_stats = {
        'mean': np.mean(z_values),
        'std': np.std(z_values),
        'min': np.min(z_values),
        'max': np.max(z_values)
    }

    return {
        'mean_neighbor_distance': mean_nn_dist,
        'depth_distribution': depth_stats,
        'num_points': len(points)
    }

# ----------------------
# Visualization Functions
# ----------------------
def draw_epipolar_lines(img1_rect, img2_rect, pts1_rect, pts2_rect, output_path):
    img1_color = cv.cvtColor(img1_rect, cv.COLOR_GRAY2BGR)
    img2_color = cv.cvtColor(img2_rect, cv.COLOR_GRAY2BGR)
    h, w = img1_rect.shape

    num_points = min(50, len(pts1_rect))
    if num_points == 0:
        return

    selected_indices = random.sample(range(len(pts1_rect)), num_points)
    pts1_selected = pts1_rect[selected_indices]
    pts2_selected = pts2_rect[selected_indices]

    for i in range(len(pts1_selected)):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        pt1 = pts1_selected[i].astype(int)
        pt2 = pts2_selected[i].astype(int)

        cv.circle(img1_color, tuple(pt1), 5, (255, 255, 255), -1)
        cv.circle(img1_color, tuple(pt1), 3, color, -1)
        cv.circle(img2_color, tuple(pt2), 5, (255, 255, 255), -1)
        cv.circle(img2_color, tuple(pt2), 3, color, -1)

        cv.line(img1_color, (0, pt1[1]), (w-1, pt1[1]), color, 1)
        cv.line(img2_color, (0, pt1[1]), (w-1, pt1[1]), color, 1)

    combined = np.hstack([img1_color, img2_color])
    cv.putText(combined, "Epipolar Lines Visualization", (10, 30), 
               cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv.imwrite(output_path, combined)


# ----------------------
# Core Reconstruction
# ----------------------
def estimate_intrinsics(ptsL, ptsR, img_shape):
    F, mask = cv.findFundamentalMat(ptsL, ptsR, cv.USAC_MAGSAC, 1.0, 0.999, 50000)
    if F is None or F.size == 0:
        raise ValueError("Fundamental matrix computation failed")
    
    h, w = img_shape
    cx, cy = w / 2, h / 2

    def compute_error(f):
        K = np.array([[f, 0, cx], [0, f, cy], [0, 0, 1]])
        E = K.T @ F @ K
        U, S, _ = np.linalg.svd(E)
        return (S[0] - S[1])**2 + S[2]**2

    res = minimize_scalar(compute_error, bounds=(0.3*w, 3*w), method='bounded')
    f_opt = res.x
    return np.array([[f_opt, 0, cx], [0, f_opt, cy], [0, 0, 1]]), F

def optimized_3d_reconstruction():
    # Load images
    img_left = cv.imread("5/Camera_Left.png", cv.IMREAD_GRAYSCALE)
    img_right = cv.imread("5/Camera_Front.png", cv.IMREAD_GRAYSCALE)
    img_left_color = cv.imread("5/Camera_Left.png", cv.IMREAD_COLOR)
    
    if img_left is None or img_right is None:
        raise FileNotFoundError("Stereo images not found")
    
    os.makedirs("output_uncalibrated", exist_ok=True)
    h, w = img_left.shape

    # Feature detection and matching
    sift = cv.SIFT_create(nfeatures=100000, contrastThreshold=0.001, edgeThreshold=5)
    kpL, desL = sift.detectAndCompute(img_left, None)
    kpR, desR = sift.detectAndCompute(img_right, None)
    
    if desL is None or desR is None or len(desL) < 100 or len(desR) < 100:
        raise ValueError("Insufficient features detected")

    # Feature matching with cross-check
    bf = cv.BFMatcher(cv.NORM_L2)
    matches1 = bf.knnMatch(desL, desR, k=2)
    matches2 = bf.knnMatch(desR, desL, k=2)
    
    good1 = [m for m, n in matches1 if m.distance < 0.8 * n.distance]
    good2 = [m for m, n in matches2 if m.distance < 0.8 * n.distance]
    
    reverse_mapping = {m.queryIdx: m.trainIdx for m in good2}
    good = [m for m in good1 if m.trainIdx in reverse_mapping and reverse_mapping[m.trainIdx] == m.queryIdx]

    if len(good) < 100:
        raise ValueError(f"Insufficient good matches: {len(good)} < 100")

    ptsL = np.float32([kpL[m.queryIdx].pt for m in good])
    ptsR = np.float32([kpR[m.trainIdx].pt for m in good])

    # Estimate intrinsics
    try:
        K, F = estimate_intrinsics(ptsL, ptsR, img_left.shape)
        print(f"Estimated intrinsic matrix:\n{K}")
    except Exception as e:
        print(f"Intrinsic estimation failed: {e}")
        K = np.array([[0.8*w, 0, w/2], [0, 0.8*w, h/2], [0, 0, 1]])
        print(f"Using fallback intrinsics:\n{K}")
        F, _ = cv.findFundamentalMat(ptsL, ptsR, cv.USAC_MAGSAC, 1.0, 0.999, 50000)

    # Evaluate epipolar error
    if F is not None:
        epipolar_error_mean, _ = compute_epipolar_error(ptsL, ptsR, F)
        
    # Estimate pose
    E, mask = cv.findEssentialMat(ptsL, ptsR, K, method=cv.RANSAC, prob=0.999, threshold=0.5, maxIters=5000)
    if E is None:
        R = np.eye(3)
        t = np.array([0, 0, 1])
        mask = np.ones((len(ptsL), 1), dtype=np.uint8)
    else:
        _, R, t, mask_pose = cv.recoverPose(E, ptsL, ptsR, K, mask=mask)
        mask = mask_pose.ravel().astype(bool)
        print(f"Recovered rotation:\n{R}\nTranslation:\n{t}")

    ptsL = ptsL[mask]
    ptsR = ptsR[mask]
    
    # Rectification
    R1, R2, P1, P2, Q, _, _ = cv.stereoRectify(
        K, None, K, None, (w, h), R, t, flags=cv.CALIB_ZERO_DISPARITY, alpha=0.9
    )

    # Create rectification maps
    mapL1, mapL2 = cv.initUndistortRectifyMap(K, None, R1, P1, (w, h), cv.CV_32FC1)
    mapR1, mapR2 = cv.initUndistortRectifyMap(K, None, R2, P2, (w, h), cv.CV_32FC1)

    # Rectify images
    img_left_rect = cv.remap(img_left, mapL1, mapL2, cv.INTER_LANCZOS4)
    img_right_rect = cv.remap(img_right, mapR1, mapR2, cv.INTER_LANCZOS4)
    
    # Rectify points
    ptsL_rect = cv.undistortPoints(ptsL.reshape(-1,1,2), K, None, R=R1, P=P1).reshape(-1,2)
    ptsR_rect = cv.undistortPoints(ptsR.reshape(-1,1,2), K, None, R=R2, P=P2).reshape(-1,2)
    
    # Epipolar line visualization
    draw_epipolar_lines(img_left_rect, img_right_rect, ptsL_rect, ptsR_rect,
                       "output_uncalibrated/rectified_epipolar.jpg")

    # Evaluate vertical disparity
    if len(ptsL_rect) > 0:
        vertical_disparity = np.abs(ptsL_rect[:, 1] - ptsR_rect[:, 1])
        mean_vertical_disp = np.mean(vertical_disparity)

    # Triangulation
    if len(ptsL) > 0:
        P0 = K @ np.hstack((np.eye(3), np.zeros((3, 1))))
        P1 = K @ np.hstack((R, t))
        pts4d = cv.triangulatePoints(P0, P1, ptsL.T, ptsR.T)
        points3d = (pts4d[:3] / pts4d[3]).T

        # Filtering
        valid_mask = fast_depth_filter(points3d)
        points3d = points3d[valid_mask]
        ptsL_valid = ptsL[valid_mask]
        ptsR_valid = ptsR[valid_mask]

    else:
        points3d = np.zeros((0, 3))
        ptsL_valid = np.zeros((0, 2))
        ptsR_valid = np.zeros((0, 2))

    # Reprojection Error Calculation
    if len(points3d) > 0:
        # Left camera reprojection
        projected_left = (K @ points3d.T).T
        projected_left = projected_left[:, :2] / projected_left[:, 2:]
        repr_error_left = np.mean(np.linalg.norm(ptsL_valid - projected_left, axis=1))

        # Right camera reprojection
        points3d_right = (R @ points3d.T + t).T
        projected_right = (K @ points3d_right.T).T
        projected_right = projected_right[:, :2] / projected_right[:, 2:]
        repr_error_right = np.mean(np.linalg.norm(ptsR_valid - projected_right, axis=1))
        avg_reproj_error = (repr_error_left + repr_error_right) / 2
    # Create colored point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points3d)
    
    if len(points3d) > 0:
        colors = get_colors(img_left_color, ptsL_valid)
        pcd.colors = o3d.utility.Vector3dVector(colors)
    
    # Point cloud cleaning
    if len(points3d) > 50:
        pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=50, std_ratio=1.5)
        pcd = pcd.voxel_down_sample(voxel_size=0.005)
    
    # Compute point cloud metrics
    pc_metrics = compute_point_cloud_metrics(pcd)
    print("\n====== Reconstruction Quality Summary ======")
    print(f"Matches used: {len(good)}")
    print(f"Epipolar Geometry Error (Pre-Rectification): {epipolar_error_mean:.4f} px")
    print(f"Vertical Disparity (Post-Rectification): {mean_vertical_disp:.4f} px")
    print(f"Average Reprojection Error: {avg_reproj_error:.4f} px")
    print(f"Point Cloud Size: {pc_metrics['num_points']}")
    print(f"Point Cloud Density: {pc_metrics['mean_neighbor_distance']:.4f} m")
    print(f"Depth Mean: {pc_metrics['depth_distribution']['mean']:.2f} m")
    print(f"Depth Range: {pc_metrics['depth_distribution']['min']:.2f} – {pc_metrics['depth_distribution']['max']:.2f} m")
    print("============================================")

    # Save point cloud
    o3d.io.write_point_cloud("output_uncalibrated/reconstruction.ply", pcd)
    print("Saved point cloud: output_uncalibrated/reconstruction.ply")

    print("Reconstruction complete. Results saved in output directory")

if __name__ == "__main__":
    optimized_3d_reconstruction()

Estimated intrinsic matrix:
[[2.61989006e+03 0.00000000e+00 9.60000000e+02]
 [0.00000000e+00 2.61989006e+03 5.40000000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Recovered rotation:
[[ 9.96319385e-01  2.87792949e-04  8.57181488e-02]
 [-2.48207380e-04  9.99999858e-01 -4.72468027e-04]
 [-8.57182725e-02  4.49453177e-04  9.96319314e-01]]
Translation:
[[-0.99996403]
 [ 0.00425277]
 [-0.00733892]]

====== Reconstruction Quality Summary ======
Matches used: 3996
Epipolar Geometry Error (Pre-Rectification): 0.0020 px
Vertical Disparity (Post-Rectification): 0.1647 px
Average Reprojection Error: 0.0922 px
Point Cloud Size: 3358
Point Cloud Density: 0.0278 m
Depth Mean: 9.40 m
Depth Range: 8.44 – 11.04 m
Saved point cloud: output_uncalibrated/reconstruction.ply
Reconstruction complete. Results saved in output directory
